In [1]:
# Building makemore Part 4: Becoming a Backprop Ninja
# https://youtu.be/q8SA3rM6ckI?si=QlPqAFqzpmKGduKW

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib

Using matplotlib backend: module://matplotlib_inline.backend_inline


In [2]:
words = open('names.txt', 'r').read().splitlines()

In [17]:
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i, s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s, i in stoi.items()}
vocab_size = len(itos)

In [18]:
block_size = 3

def build_dataset(words):
    X, Y = [], []

    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = stoi[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(X.shape, Y.shape)
    return X, Y

import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

torch.Size([182580, 3]) torch.Size([182580])
torch.Size([22767, 3]) torch.Size([22767])
torch.Size([22799, 3]) torch.Size([22799])


In [19]:
def cmp(s, dt, t):
    ex = torch.all(dt == t.grad).item()
    # approximate
    app = torch.allclose(dt, t.grad)
    maxdiff = (dt - t.grad).abs().max().item()
    print(f'{s:15s} | exact: {str(ex):5s} | approximate: {str(app):5s} | maxdiff: {maxdiff}')

In [20]:
n_embd = 10 # embd dim
n_hidden = 64 # neurons in hidden layer

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)

In [21]:
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) # not necessary, we have batch norm

In [22]:
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

In [23]:
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

In [24]:
# These initializations are non-standard, because doing all zeros could mask
# an incorrect implementation of the backward pass

In [25]:
parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))

4137


In [26]:
for p in parameters:
    p.requires_grad = True

In [27]:
n = 32 # batch_size
ix = torch.randint(0, Xtr.shape[0], (n,), generator=g)
Xb, Yb = Xtr[ix], Ytr[ix]

In [28]:
# Forward pass
emb = C[Xb]
embcat = emb.view(emb.shape[0], -1)

# Linear layer 1
hprebn = embcat @ W1 + b1 # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani # deviation from mean, matrix
bndiff2 = bndiff ** 2
bnvar = 1 / (n - 1) * (bndiff2).sum(0, keepdim=True) # n-1: Bessel's correction, unbiased
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# Non-linearity
h = torch.tanh(hpreact) # hidden layer
# Linear layer 2
logits = h @ W2 + b2 # output layer
# Cross-entropy loss (same as F.cross_entropy(logits, Yb))
logit_maxes = logits.max(1, keepdim=True).values
norm_logits = logits - logit_maxes # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
counts_sum_inv = counts_sum ** -1 # if (1.0 / counts_sum), backprop won't be bit exact
probs = counts * counts_sum_inv
logprobs = probs.log()
# going down the rows - range(n) - and always plucking out index in Yb 
loss = -logprobs[range(n), Yb].mean()

In [29]:
# pytorch backward pass
for p in parameters:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,
          norm_logits, logit_maxes, logits, h, hpreact, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hprebn, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.4899, grad_fn=<NegBackward0>)

In [30]:
# Exercise 1: backprop through whole thing manually,
# backpropagating through exactly all of the variables
# as they are defined in the forward pass above, one by one

# dlogprobs: derivative of loss in relation to logprobs
dlogprobs = torch.zeros_like(logprobs) # keep shape
dlogprobs[range(n), Yb] = - 1.0 / n # derivative of mean
cmp('logprobs', dlogprobs, logprobs)

# dprobs
dprobs = torch.zeros_like(probs)
dprobs = (1.0 / probs) * dlogprobs # chain rule
cmp('probs', dprobs, probs)

# dcounts_sum_inv
# remember: counts_sum_inv was replicated, and this is the derivative with respect
# to the replicated vectors
# counts * dprobs is deriv in relation to replicated counts_sum_inv
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True) # keep dim of counts_sum_inv
cmp('counts_sum_inv', dcounts_sum_inv, counts_sum_inv)

# dcounts
dcounts = counts_sum_inv * dprobs

# dcounts_sum
dcounts_sum = (-counts_sum**-2) * dcounts_sum_inv
cmp('counts_sum', dcounts_sum, counts_sum)

# dcounts
dcounts += torch.ones_like(counts) * dcounts_sum # broadcasting
cmp('counts', dcounts, counts)

# dnorm_logits
dnorm_logits = (counts) * dcounts # counts = norm_logits.exp()
cmp('norm_logits', dnorm_logits, norm_logits)

# dlogits
dlogits = dnorm_logits.clone() # counts = norm_logits.exp()

# dlogit_maxes
dlogit_maxes = (-dnorm_logits).sum(1, keepdim=True)  # counts = norm_logits.exp()
cmp('logit_maxes', dlogit_maxes, logit_maxes)

# dlogits
dlogits += F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_maxes
cmp('logits', dlogits, logits)

# dh
# shape of dh must be same as shape of h (from matmul)
dh = dlogits @ W2.T
cmp('h', dh, h)

# dW2
dW2 = h.T @ dlogits
cmp('W2', dW2, W2)

# db2
db2 = dlogits.sum(0)
cmp('b2', db2, b2)

# dhpreact
dhpreact = (1 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# dhpreact
dhpreact = (1 - h**2) * dh
cmp('hpreact', dhpreact, hpreact)

# dbngain, dbnraw and dbnbias can be deduced using dimensions
# dbngain
dbngain = (bnraw * dhpreact).sum(0, keepdim=True) # dbngain and bngain have equal dimensions [1, 64]
# bngain [1, 64]
# bnraw [32, 64]
# dhpreact [32, 64]
# bngain * bnraw is element-wise
# bngain is replicated across rows
# to account for that, we sum across dimension 0 (rows)
# similar to docunts_sum_inv
cmp('bngain', dbngain, bngain)

# dbnraw
dbnraw = bngain * dhpreact
cmp('bnraw', dbnraw, bnraw)

# dbnbias
dbnbias = dhpreact.sum(0, keepdim=True)
cmp('bnbias', dbnbias, bnbias)

# dbndiff
dbndiff = bnvar_inv * dbnraw

# dbnvar_inv
dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)

# dbnvar
dbnvar = -0.5*(bnvar + 1e-5)**-1.5 * dbnvar_inv
cmp('bnvar', dbnvar, bnvar)

# dbndiff2
# a11 a12
# a21 a22
# ----->
# b1  b2, where:
# b1 = 1 / (n-1) * (a11 + a21)
# b2 = 1 / (n-1) * (a21 + a22)
dbndiff2 = (1.0/(n-1)) * torch.ones_like(bndiff2) * dbnvar
cmp('bndiff2', dbndiff2, bndiff2)

# dbndiff
dbndiff += (2 * bndiff) * dbndiff2
cmp('bndiff', dbndiff, bndiff)

# dhprebn
dhprebn = dbndiff.clone()

# dbnmeani
dbnmeani = (-dbndiff).sum(0)
cmp('bnmeani', dbnmeani, bnmeani)

# dhprebn
dhprebn += 1.0 / n * (torch.ones_like(hprebn) * dbnmeani)
cmp('hprebn', dhprebn, hprebn)

# dembcat
dembcat = dhprebn @ W1.T
cmp('embcat', dembcat, embcat)

# dW1
dW1 = embcat.T @ dhprebn
cmp('W1', dW1, W1)

# db1
db1 = dhprebn.sum(0)
cmp('b1', db1, b1)

# demb
demb = dembcat.view(emb.shape)
cmp('emb', demb, emb)

# dC
dC = torch.zeros_like(C) # [27, 10]
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        ix = Xb[k, j]
        dC[ix] += demb[k, j]
cmp('dC', dC, C)

logprobs        | exact: True  | approximate: True  | maxdiff: 0.0
probs           | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum_inv  | exact: True  | approximate: True  | maxdiff: 0.0
counts_sum      | exact: True  | approximate: True  | maxdiff: 0.0
counts          | exact: True  | approximate: True  | maxdiff: 0.0
norm_logits     | exact: True  | approximate: True  | maxdiff: 0.0
logit_maxes     | exact: True  | approximate: True  | maxdiff: 0.0
logits          | exact: True  | approximate: True  | maxdiff: 0.0
h               | exact: True  | approximate: True  | maxdiff: 0.0
W2              | exact: True  | approximate: True  | maxdiff: 0.0
b2              | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
hpreact         | exact: True  | approximate: True  | maxdiff: 0.0
bngain          | exact: True  | approximate: True  | maxdiff: 0.0
bnraw           | exact: True  | approximate: True  | maxdiff:

In [31]:
# sum in forward pass: replication/broadcasting in backward pass
# replication/broadcast in forward pass: sum in backward pass

In [32]:
# Exercise 2: backprop through cross-entropy in one go
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.489867925643921 diff: 2.384185791015625e-07


In [33]:
# backward pass
dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n
cmp('dlogits', dlogits, logits)

dlogits         | exact: False | approximate: True  | maxdiff: 6.05359673500061e-09


In [34]:
# Exercise 3: backprop through batchnorm in one go
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max_diff:', (hpreact_fast - hpreact).abs().max())

max_diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [35]:
# backward pass
dhprebn = bngain * bnvar_inv / n * (n * dhpreact - dhpreact.sum(0) - n / (n - 1) * bnraw * (dhpreact * bnraw).sum(0))
cmp('hprebn', dhprebn, hprebn)

hprebn          | exact: False | approximate: True  | maxdiff: 9.313225746154785e-10


In [74]:
# Exercise 4: putting it all together
# train NN using my own backward pass

# init
n_embd = 10
n_hidden = 200

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.randn((1, n_hidden)) * 0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True

# same optim as last time
max_steps = 10000
batch_size = 32
n = batch_size
lossi = []

with torch.no_grad():
    # optim
    for i in range(max_steps):
    
        # minibatch construct
        ix = torch.randint(0, Xtr.shape[0], (batch_size,), generator=g)
        Xb, Yb = Xtr[ix], Ytr[ix]
    
        # forward pass
        emb = C[Xb]
        embcat = emb.view(Xb.shape[0], -1)
        # Linear Layer
        hprebn = embcat @ W1 + b1
        # BatchNorm
        bnmean = hprebn.mean(0, keepdim=True)
        bnvar = hprebn.var(0, keepdim=True, unbiased=True)
        bnvar_inv = (bnvar + 1e-5) ** -0.5
        bnraw = (hprebn - bnmean) * bnvar_inv
        hpreact = bngain * bnraw + bnbias
        # Non-linearity
        h = torch.tanh(hpreact) # hidden layer
        logits = h @ W2 + b2 # output layer
        loss = F.cross_entropy(logits, Yb) # loss function
    
        # backward pass
        for p in parameters:
            p.grad = None
        # loss.backward()
    
        dC, dW1, db1, dW2, db2, dbngain, dbnbias = None, None, None, None, None, None, None
        # manual backprop
        # Cross Entropy Layer
        dlogits = F.softmax(logits, 1)
        dlogits[range(n), Yb] -= 1
        dlogits /= n
        # 2nd Layer
        dh = dlogits @ W2.T
        dW2 = h.T @ dlogits
        db2 = dlogits.sum(0)
        # Tanh
        dhpreact = (1.0 - h**2) * dh
        # BatchNorm
        dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
        dbnbias = dhpreact.sum(0, keepdim=True)
        dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
        # 1st layer
        dembcat = dhprebn @ W1.T
        dW1 = embcat.T @ dhprebn
        db1 = dhprebn.sum(0)
        # Embedding
        demb = dembcat.view(emb.shape)
        dC = torch.zeros_like(C)
        for k in range(Xb.shape[0]):
          for j in range(Xb.shape[1]):
            ix = Xb[k,j]
            dC[ix] += demb[k,j]
        grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    
        # update
        lr = 0.1 if i < 10000 else 0.01
        for p, grad in zip(parameters, grads):
            # p.data += -lr * p.grad # old way
            p.data += -lr * grad
    
        # track stats
        if i % 1000 == 0:
            print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
        lossi.append(loss.log10().item())
    
        if i >= 5000:
            break

12297
      0/  10000: 3.6105
   1000/  10000: 2.4734
   2000/  10000: 2.2887
   3000/  10000: 2.1313
   4000/  10000: 1.9494
   5000/  10000: 2.6141


In [75]:
# for p, g in zip(parameters, grads):
#     cmp(str(tuple(p.shape)), g, p)

In [76]:
# calibrate batch norm at end of training
with torch.no_grad():
    # pass training set through
    emb = C[Xtr]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    # measure mean/std
    bnmean = hpreact.mean(0, keepdim=True)
    bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [77]:
# evaluate train and val loss
@torch.no_grad()
def split_loss(split):
    x, y = {
        'train': (Xtr, Ytr),
        'val': (Xdev, Ydev),
        'test': (Xtest, Ytest)
    }[split]
    emb = C[x]
    embcat = emb.view(emb.shape[0], -1)
    hpreact = embcat @ W1 + b1
    hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5) ** -0.5 + bnbias
    h = torch.tanh(hpreact) 
    logits = h @ W2 + b2
    loss = F.cross_entropy(logits, y)
    print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.3058314323425293
val 2.314452886581421


In [78]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(itos[i] for i in out))

carpahfato.
lle.
filmrix.
laty.
halani.
ejanonel.
dellah.
jareei.
nellana.
chaiiv.
kaleig.
dham.
poin.
qhinn.
sulin.
aniveni.
wattrogiearyni.
fice.
pilsai.
ed.
